# Tag 4: Wahrnehmung, Anwendung & Transfer in Projekte
## Unitree G1 Humanoid Roboter Academy

### Agenda Tag 4:
1. Visuelle Wahrnehmung & Deep Learning
2. Objekt- und Umgebungserkennung
3. Multi-Sensor-Fusion
4. Vollständige System-Integration (Geoff-Stack)
5. Projektarbeit: Real-World Challenge
6. Transfer: "Wie setze ich es im Unternehmen um?"
7. Abschlusspräsentationen

---
## 1. Visuelle Wahrnehmung mit Deep Learning

### 1.1 Warum Visuelle Wahrnehmung?

**LiDAR alleine ist nicht genug:**
- ✓ Sehr genaue 3D-Geometrie
- ✗ Keine Farb-/Textur-Information
- ✗ Keine semantische Information (Was ist ein Tisch? Eine Tür?)

**RGB-Kameras + Deep Learning:**
- ✓ Objekterkennung ("Das ist eine Tasse")
- ✓ Szenenverständnis ("Das ist ein Büro")
- ✓ Texterkennung (OCR)
- ✓ Personenerkennung, Gestenerkennung

**Kombination LiDAR + RGB:**
- **3D Bounding Boxes** um erkannte Objekte
- **Semantisches Mapping** ("Hier ist der Schreibtisch")
- **Robuste Navigation** (Geometrie + Semantik)

### 1.2 Objekterkennung mit YOLO

**YOLO (You Only Look Once)** - Echtzeit-Objekterkennung

**Setup:**

In [ ]:
# Installation (falls nicht vorhanden):
# pip install ultralytics opencv-python

from ultralytics import YOLO
import cv2
import numpy as np

# Lade vortrainiertes Modell
# model = YOLO('yolov8n.pt')  # Nano (schnell)
# model = YOLO('yolov8s.pt')  # Small
# model = YOLO('yolov8m.pt')  # Medium (empfohlen für G1)

print("YOLO Setup:")
print("- yolov8n: ~3ms/frame (Jetson)")
print("- yolov8s: ~8ms/frame")
print("- yolov8m: ~20ms/frame")
print("\nModell erkennt 80 COCO-Klassen:")
print("Person, Stuhl, Tisch, Laptop, Tasse, Flasche, ...")

**Live-Detection auf RealSense Stream:**

In [ ]:
# Beispiel: Objekterkennung auf RGB-Frame
def detect_objects_in_frame(rgb_frame, model):
    """
    Führt YOLO-Detection auf RGB-Frame aus
    
    rgb_frame: numpy array (H, W, 3)
    model: YOLO Modell
    
    Returns: Liste von Detektionen
    """
    # Inference
    results = model(rgb_frame, verbose=False)
    
    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Extrahiere Info
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            confidence = box.conf[0].cpu().numpy()
            class_id = int(box.cls[0].cpu().numpy())
            class_name = model.names[class_id]
            
            detections.append({
                'bbox': (x1, y1, x2, y2),
                'confidence': confidence,
                'class': class_name
            })
    
    return detections

def draw_detections(frame, detections):
    """
    Zeichnet Bounding Boxes auf Frame
    """
    for det in detections:
        x1, y1, x2, y2 = det['bbox']
        conf = det['confidence']
        cls = det['class']
        
        # Zeichne Box
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), 
                     (0, 255, 0), 2)
        
        # Label
        label = f"{cls} {conf:.2f}"
        cv2.putText(frame, label, (int(x1), int(y1)-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return frame

print("Objekterkennungs-Funktionen definiert.")
print("\nVerwendung:")
print("detections = detect_objects_in_frame(rgb_frame, model)")
print("annotated_frame = draw_detections(rgb_frame, detections)")

### 1.3 3D Object Localization (RGB-D)

**Ziel:** Von 2D Bounding Box → 3D Position im Raum

In [ ]:
import numpy as np

def get_3d_position_from_bbox(bbox, depth_frame, camera_intrinsics):
    """
    Berechnet 3D-Position eines Objekts aus 2D BBox + Tiefenbild
    
    bbox: (x1, y1, x2, y2)
    depth_frame: Tiefenbild (H, W) in Metern
    camera_intrinsics: Dict mit fx, fy, cx, cy
    
    Returns: (x, y, z) in Kamera-Koordinaten
    """
    x1, y1, x2, y2 = bbox
    
    # Zentrum der BBox
    center_x = int((x1 + x2) / 2)
    center_y = int((y1 + y2) / 2)
    
    # Tiefe am Zentrum (Median für Robustheit)
    depth_patch = depth_frame[
        int(y1):int(y2),
        int(x1):int(x2)
    ]
    
    # Filtere Null-Werte
    valid_depths = depth_patch[depth_patch > 0]
    if len(valid_depths) == 0:
        return None
    
    depth = np.median(valid_depths)
    
    # Deprojection (Pixel → 3D)
    fx = camera_intrinsics['fx']
    fy = camera_intrinsics['fy']
    cx = camera_intrinsics['cx']
    cy = camera_intrinsics['cy']
    
    x_3d = (center_x - cx) * depth / fx
    y_3d = (center_y - cy) * depth / fy
    z_3d = depth
    
    return (x_3d, y_3d, z_3d)

# RealSense D435i Intrinsics (typisch)
realsense_intrinsics = {
    'fx': 615.0,
    'fy': 615.0,
    'cx': 320.0,
    'cy': 240.0
}

# Beispiel:
example_bbox = (100, 150, 200, 300)  # Pixel
example_depth = np.random.rand(480, 640) * 3.0  # Simuliert

pos_3d = get_3d_position_from_bbox(example_bbox, example_depth, realsense_intrinsics)
if pos_3d:
    print(f"Objekt-Position: x={pos_3d[0]:.2f}m, y={pos_3d[1]:.2f}m, z={pos_3d[2]:.2f}m")
    print("(relativ zur Kamera)")

---
## 2. Semantisches Mapping

### 2.1 Konzept

**Klassische Karte:**
```
Grid: [0, 0, 1, 1, 0, ...]
      (frei, frei, belegt, belegt, frei)
```

**Semantische Karte:**
```
Grid: [(x, y): "Stuhl",
       (x, y): "Tisch",
       (x, y): "Tür",
       ...]
```

**Vorteil:**
- High-Level Navigation: "Gehe zum Tisch"
- Kontext-bewusste Planung: "Vermeide Stühle, aber nutze Türen"
- Besseres Szenenverständnis

In [ ]:
# Semantic Map Datenstruktur
class SemanticMap:
    def __init__(self):
        self.objects = []  # Liste von semantischen Objekten
    
    def add_object(self, class_name, position_3d, bbox_2d=None):
        """
        Fügt Objekt zur Karte hinzu
        
        class_name: z.B. "chair", "table"
        position_3d: (x, y, z) in Weltkoordinaten
        bbox_2d: Optional, für Visualisierung
        """
        obj = {
            'class': class_name,
            'position': position_3d,
            'bbox': bbox_2d,
            'timestamp': time.time()
        }
        
        # Prüfe ob Objekt bereits existiert (Duplikate vermeiden)
        for existing in self.objects:
            if (existing['class'] == class_name and
                np.linalg.norm(np.array(existing['position']) - 
                              np.array(position_3d)) < 0.5):  # 50cm Toleranz
                # Update statt neu hinzufügen
                existing['position'] = position_3d
                existing['timestamp'] = time.time()
                return
        
        self.objects.append(obj)
    
    def find_objects_by_class(self, class_name):
        """Finde alle Objekte einer bestimmten Klasse"""
        return [obj for obj in self.objects if obj['class'] == class_name]
    
    def find_nearest_object(self, class_name, robot_position):
        """Finde nächstes Objekt einer Klasse"""
        candidates = self.find_objects_by_class(class_name)
        if not candidates:
            return None
        
        nearest = min(candidates, 
                     key=lambda obj: np.linalg.norm(
                         np.array(obj['position'][:2]) - 
                         np.array(robot_position)))
        return nearest

# Test:
import time
semantic_map = SemanticMap()
semantic_map.add_object('chair', (2.0, 1.0, 0.5))
semantic_map.add_object('table', (3.0, 0.0, 0.8))
semantic_map.add_object('chair', (2.5, -1.0, 0.5))

print(f"Karte enthält {len(semantic_map.objects)} Objekte")
chairs = semantic_map.find_objects_by_class('chair')
print(f"Davon {len(chairs)} Stühle")

robot_pos = (0.0, 0.0)
nearest_chair = semantic_map.find_nearest_object('chair', robot_pos)
if nearest_chair:
    print(f"Nächster Stuhl bei: {nearest_chair['position']}")

---
## 3. Multi-Sensor-Fusion

### 3.1 Sensor-Komplementarität

| Sensor | Stärken | Schwächen |
|--------|---------|----------|
| **LiDAR** | Präzise 3D-Geometrie, große Reichweite | Keine Farbe/Semantik, teuer |
| **RGB** | Farbe, Textur, Semantik (DL) | Lichtabhängig, keine Tiefe |
| **Depth (RealSense)** | Tiefe + RGB, günstig | Begrenzte Reichweite (~10m) |
| **IMU** | Schnelle Orientierung, drift | Akkumulierter Fehler, keine Position |

**Fusion-Strategie:**
- **LiDAR**: Globale Karte, präzise Lokalisierung
- **RGB-D**: Objekterkennung, nahe Hindernisvermeidung
- **IMU**: Schnelle Orientierungs-Updates zwischen LiDAR-Scans

### 3.2 Koordinatentransformationen

**Problem:** Sensoren haben unterschiedliche Koordinatensysteme

```
         Base (Roboter)
              |
      ┌───────┴───────┐
      │               │
   LiDAR          RealSense
   (0, 0, 0.3)    (0.1, 0, 0.25)
```

**Lösung: TF Tree (Transformation Tree)**

In [ ]:
import numpy as np

class TransformTree:
    def __init__(self):
        # Statische Transformationen (fest montiert)
        self.transforms = {
            'base_to_lidar': np.array([
                [1, 0, 0, 0.0],
                [0, 1, 0, 0.0],
                [0, 0, 1, 0.3],  # 30cm über Base
                [0, 0, 0, 1]
            ]),
            'base_to_realsense': np.array([
                [1, 0, 0, 0.1],   # 10cm nach vorne
                [0, 1, 0, 0.0],
                [0, 0, 1, 0.25],  # 25cm hoch
                [0, 0, 0, 1]
            ])
        }
    
    def transform_point(self, point, from_frame, to_frame):
        """
        Transformiert Punkt von einem Frame zu anderem
        
        point: (x, y, z)
        from_frame: z.B. 'realsense'
        to_frame: z.B. 'base' oder 'lidar'
        """
        # Konvertiere zu homogenen Koordinaten
        point_homo = np.array([point[0], point[1], point[2], 1.0])
        
        # Transformation
        if from_frame == 'realsense' and to_frame == 'base':
            # Inverse Transformation
            T_inv = np.linalg.inv(self.transforms['base_to_realsense'])
            point_transformed = T_inv @ point_homo
        elif from_frame == 'base' and to_frame == 'realsense':
            point_transformed = self.transforms['base_to_realsense'] @ point_homo
        else:
            # Generisch: via base frame
            # from_frame → base → to_frame
            pass
        
        return point_transformed[:3]

# Test:
tf_tree = TransformTree()

# Objekt bei (1.0, 0.0, 0.5) in RealSense-Frame
obj_realsense = (1.0, 0.0, 0.5)
obj_base = tf_tree.transform_point(obj_realsense, 'realsense', 'base')

print(f"Objekt in RealSense-Frame: {obj_realsense}")
print(f"Objekt in Base-Frame: {obj_base}")

---
## 4. Geoff-Stack: Vollständige Integration

### 4.1 Geoff-Stack Architektur

Das Repository enthält **Geoff-Stack** - eine vollständige GUI-basierte Integration:

```
┌─────────────────────────────────────────────────┐
│          run_geoff_gui.py (PySide6)             │
├──────────────┬────────────┬─────────────────────┤
│ RGB View     │ Depth View │ 2D SLAM Map         │
│ (640x480)    │ (Colorized)│ (Bird's Eye)        │
│              │            │                     │
│  [Live       │  [Live     │  [Occupancy Grid]   │
│   Camera]    │   Depth]   │                     │
├──────────────┴────────────┴─────────────────────┤
│         3D Point Cloud (pyqtgraph)              │
│         [Interactive 3D Visualization]          │
├─────────────────────────────────────────────────┤
│  Status: vx=0.2 vy=0.0 ω=0.0 | Battery: 85%    │
└─────────────────────────────────────────────────┘
```

**Background Threads:**
1. **RealSense RX**: Empfängt RGB+Depth via GStreamer
2. **SLAM Thread**: KISS-ICP Verarbeitung
3. **Battery Monitor**: Liest Akku-Status
4. **Keyboard Controller**: WASD-Steuerung
5. **GUI Render**: 30 Hz Update

### 4.2 Geoff-GUI starten

In [ ]:
# Im Terminal:
# cd ../unitree_g1_vibes
# python3 run_geoff_gui.py --iface enp68s0f1

print("Geoff-GUI Features:")
print("====================")
print("✓ Vier-Panel-Ansicht (RGB, Depth, 2D Map, 3D Cloud)")
print("✓ Echtzeit SLAM-Visualisierung")
print("✓ Keyboard-Steuerung integriert (WASD)")
print("✓ Batterie-Monitor")
print("✓ FPS-Overlay")
print("✓ Thread-basiert, non-blocking")
print("\nStart:")
print("python3 run_geoff_gui.py --iface <network_interface>")

### 4.3 Eigene Integration erweitern

**Beispiel: YOLO in Geoff-GUI integrieren**

In [ ]:
# Pseudo-Code: YOLO Integration
# (Basierend auf run_geoff_gui.py)

from ultralytics import YOLO
import threading

class GeoffGuiExtended:
    def __init__(self):
        # ... existing setup ...
        
        # YOLO hinzufügen
        self.yolo_model = YOLO('yolov8m.pt')
        self.detections = []
        self.detection_lock = threading.Lock()
        
        # Detection Thread
        self.detection_thread = threading.Thread(
            target=self.detection_loop,
            daemon=True
        )
        self.detection_thread.start()
    
    def detection_loop(self):
        """Läuft mit ~10 Hz (nicht bei jedem Frame)"""
        while self.running:
            # Hole aktuellen RGB-Frame
            with self.rgb_lock:
                rgb_frame = self.rgb_frame.copy()
            
            # YOLO Inference
            results = self.yolo_model(rgb_frame, verbose=False)
            
            # Extrahiere Detektionen
            detections = []
            for result in results:
                for box in result.boxes:
                    # ... extract info ...
                    detections.append({...})
            
            # Update shared state
            with self.detection_lock:
                self.detections = detections
            
            time.sleep(0.1)  # 10 Hz
    
    def render_rgb_panel(self):
        """Überschreibe Render-Funktion"""
        # ... get rgb_frame ...
        
        # Zeichne Detektionen
        with self.detection_lock:
            for det in self.detections:
                # Draw bounding box
                cv2.rectangle(rgb_frame, ...)
        
        # ... update Qt widget ...

print("YOLO kann in Geoff-GUI als zusätzlicher Thread integriert werden.")
print("Wichtig: Lock-Protected Shared State für Thread-Safety!")

---
## 5. Projektarbeit: Real-World Challenge

### 5.1 Challenge-Beschreibung

**Titel: "Autonomer Büro-Assistent"**

**Aufgabe:**
Der G1 soll autonom:
1. Einen Raum scannen und Karte erstellen
2. Bestimmte Objekte finden (z.B. "Finde eine Tasse")
3. Zum Objekt navigieren
4. Greifbewegung ausführen (simuliert oder real)
5. Zurück zu Startposition

**Technologien:**
- SLAM (KISS-ICP)
- Objekterkennung (YOLO)
- Pfadplanung (A*)
- Lokale Steuerung (DWA)
- Arm-RL-Policy (optional)

### 5.2 Implementierungs-Template

In [ ]:
# Challenge Implementation Template

class OfficeAssistantChallenge:
    def __init__(self, bot, slam, yolo_model):
        self.bot = bot
        self.slam = slam
        self.yolo = yolo_model
        self.semantic_map = SemanticMap()
    
    def run_challenge(self, target_object='cup'):
        """
        Hauptschleife der Challenge
        """
        print(f"=== Starte Challenge: Finde '{target_object}' ===")
        
        # Phase 1: Exploration & Mapping
        print("\n[Phase 1] Raum scannen...")
        self.explore_room(duration=30.0)
        
        # Phase 2: Object Detection
        print(f"\n[Phase 2] Suche nach '{target_object}'...")
        target_pos = self.find_object(target_object)
        
        if target_pos is None:
            print(f"✗ '{target_object}' nicht gefunden!")
            return False
        
        print(f"✓ '{target_object}' gefunden bei {target_pos}")
        
        # Phase 3: Navigation
        print(f"\n[Phase 3] Navigiere zu {target_pos}...")
        success = self.navigate_to_position(target_pos)
        
        if not success:
            print("✗ Navigation fehlgeschlagen!")
            return False
        
        # Phase 4: Manipulation (optional)
        print("\n[Phase 4] Greife Objekt...")
        # self.reach_and_grasp(target_pos)
        print("(Greifen simuliert)")
        
        # Phase 5: Return Home
        print("\n[Phase 5] Kehre zurück zu Start...")
        self.navigate_to_position((0, 0, 0))
        
        print("\n=== Challenge abgeschlossen! ===")
        return True
    
    def explore_room(self, duration):
        """
        Frontier-based Exploration
        """
        start_time = time.time()
        
        while time.time() - start_time < duration:
            # 1. Update SLAM
            # points = lidar.get_latest_points()
            # pose = slam.register_frame(points)
            
            # 2. Run Object Detection
            # rgb_frame = realsense.get_rgb()
            # detections = yolo(rgb_frame)
            # for det in detections:
            #     pos_3d = get_3d_position(...)
            #     self.semantic_map.add_object(det['class'], pos_3d)
            
            # 3. Simple Exploration Strategy
            # bot.Move(vx=0.1, vy=0.0, omega=0.2)  # Spirale
            
            time.sleep(0.1)
        
        # bot.StopMove()
        print(f"Exploration abgeschlossen. {len(self.semantic_map.objects)} Objekte gefunden.")
    
    def find_object(self, class_name):
        """
        Sucht Objekt in semantischer Karte
        """
        objects = self.semantic_map.find_objects_by_class(class_name)
        if not objects:
            return None
        
        # Wähle nächstes
        robot_pos = (0, 0)  # TODO: Von SLAM holen
        nearest = self.semantic_map.find_nearest_object(class_name, robot_pos)
        return nearest['position']
    
    def navigate_to_position(self, goal_pos):
        """
        Navigation mit A* + DWA
        """
        # (Siehe Tag 3 für Details)
        print(f"Navigation zu {goal_pos} (implementiert in Tag 3)")
        return True

print("Challenge-Template definiert.")
print("\nVerwendung:")
print("challenge = OfficeAssistantChallenge(bot, slam, yolo_model)")
print("challenge.run_challenge(target_object='cup')")

### 5.3 Team-Aufgaben

**Teilen Sie sich in 3-4er Teams auf:**

**Team 1: Perception**
- Optimiere YOLO-Performance
- Implementiere 3D Object Localization
- Baue Semantische Karte auf

**Team 2: Navigation**
- Implementiere robuste Pfadplanung
- Tune DWA-Parameter
- Handle Edge-Cases (kein Pfad, Deadlocks)

**Team 3: Integration**
- Verbinde alle Komponenten
- Baue State Machine für Challenge-Ablauf
- Erstelle GUI/Monitoring

**Team 4: Manipulation (optional)**
- Trainiere/tune RL-Policy für Arm
- Implementiere Greif-Logik
- Teste Sim-to-Real Transfer

---
## 6. Transfer: "Wie setze ich es im Unternehmen um?"

### 6.1 Von Prototyp zu Produktion

**Prototyp (Academy):**
- Einzelner Roboter
- Manuelle Starts
- Entwickler-Umgebung
- Jupyter Notebooks

**Produktion (Unternehmen):**
- Flotte von Robotern
- Automatisierte Deployment
- 24/7 Betrieb
- Monitoring & Logging

### 6.2 Deployment-Checkliste

**1. Software-Engineering:**
```python
# Von Notebook zu Modul
# Bad:
# - Code in .ipynb
# - Globale Variablen
# - Keine Tests

# Good:
# - Strukturierte Packages (src/perception, src/navigation, ...)
# - Unit Tests (pytest)
# - CI/CD Pipeline (GitHub Actions)
# - Docker Container
```

**2. Monitoring & Logging:**
```python
import logging

# Setup centralized logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(name)s] %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler('/var/log/g1_robot.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger('g1.navigation')
logger.info("Started navigation to waypoint (2.0, 3.0)")
```

**3. Fehlerbehandlung:**
```python
# Robust gegen Fehler
try:
    path = astar_planning(grid, start, goal)
except Exception as e:
    logger.error(f"Path planning failed: {e}")
    # Fallback: Return to safe position
    bot.navigate_to_home()
    # Alert operator
    send_alert("Navigation failure", severity="high")
```

**4. Configuration Management:**
```yaml
# config.yaml
robot:
  id: "G1-001"
  network_interface: "enp68s0f1"

perception:
  yolo_model: "yolov8m.pt"
  confidence_threshold: 0.6

navigation:
  max_velocity: 0.5
  safety_margin: 0.3
  planner: "astar"
```

**5. Remote Monitoring:**
```python
# Metrics export (Prometheus/Grafana)
from prometheus_client import Counter, Gauge, start_http_server

navigation_success = Counter('navigation_success_total', 'Successful navigations')
battery_level = Gauge('battery_level_percent', 'Current battery level')
cpu_usage = Gauge('cpu_usage_percent', 'CPU usage')

# Start metrics server
start_http_server(8000)
```

### 6.3 Use-Cases im Unternehmen

**Logistik & Warehousing:**
- Inventur-Automatisierung (Regale scannen)
- Waren-Transport (Pick & Place)
- Lager-Mapping

**Inspektion & Wartung:**
- Anlageninspektion (Ventile ablesen, Lecks erkennen)
- Routinekontrollen (täglich gleichr Rundgang)
- Dokumentation (Fotos, Messwerte sammeln)

**Service & Kundeninteraktion:**
- Empfangsroboter (Gäste begrüßen, Führungen)
- Informations-Terminal (Fragen beantworten)
- Lieferservice (innerhalb Gebäude)

**Forschung & Entwicklung:**
- Plattform für Algorithmen-Entwicklung
- Benchmark-Tests (Vergleich verschiedener Ansätze)
- Sim-to-Real Validierung

### 6.4 ROI-Kalkulation

**Beispiel: Inventur-Automatisierung**

**Kosten:**
- Unitree G1: ~€80,000
- Entwicklung (3 Monate): €30,000
- Wartung (jährlich): €5,000
- **Total (Jahr 1)**: €115,000

**Einsparungen:**
- 2 Mitarbeiter à €40,000/Jahr = €80,000
- Zeitersparnis (24/7 Betrieb): +30% Durchsatz
- Fehlerreduktion: -20% Fehlbestände → €20,000/Jahr
- **Total Einsparungen**: €100,000/Jahr

**Break-Even:** ~14 Monate

**Wichtig:**
- Realistische Erwartungen
- Pilotphase einplanen
- Edge-Cases berücksichtigen
- Training & Support einkalkulieren

---
## 7. Best Practices & Lessons Learned

### 7.1 Technische Best Practices

**1. Modularität:**
```python
# Good: Austauschbare Komponenten
class Planner(ABC):
    @abstractmethod
    def plan(self, start, goal):
        pass

class AStarPlanner(Planner):
    def plan(self, start, goal):
        # A* implementation
        pass

class RRTPlanner(Planner):
    def plan(self, start, goal):
        # RRT implementation
        pass

# Einfach austauschbar:
planner = AStarPlanner()  # oder RRTPlanner()
```

**2. Fail-Safe Design:**
- Immer Fallback-Strategien
- Watchdogs für kritische Threads
- Graceful Degradation (lieber langsam als Absturz)

**3. Testing:**
```python
# Unit Tests
def test_astar_simple_path():
    grid = np.zeros((100, 100))
    start = (10, 10)
    goal = (90, 90)
    path = astar_planning(grid, start, goal)
    assert path is not None
    assert path[0] == start
    assert path[-1] == goal

# Integration Tests
def test_full_navigation_pipeline():
    # Test entire chain: SLAM → Planning → Control
    pass
```

**4. Documentation:**
```python
def navigate_to_goal(bot, slam, goal, max_retries=3):
    """
    Navigiert Roboter zu Zielpunkt mit Retry-Logik.
    
    Args:
        bot (LocoClient): Roboter-Steuerung
        slam (KissICP): SLAM-Instanz für Lokalisierung
        goal (tuple): (x, y) Zielposition in Metern
        max_retries (int): Maximale Wiederholungen bei Fehler
    
    Returns:
        bool: True wenn erfolgreich, False sonst
    
    Raises:
        NavigationError: Bei kritischen Fehlern
    
    Example:
        >>> success = navigate_to_goal(bot, slam, (5.0, 3.0))
        >>> if success:
        ...     print("Ziel erreicht")
    """
    pass
```

### 7.2 Organizational Best Practices

**1. Cross-Functional Teams:**
- Robotik-Ingenieure
- Software-Entwickler
- Domain-Experten (z.B. Logistik)
- Safety-Officers

**2. Iterative Development:**
- Sprint 1: Basic Movement
- Sprint 2: SLAM Integration
- Sprint 3: Object Detection
- Sprint 4: Full Navigation
- Sprint 5: Error Handling
- Sprint 6: Production Deployment

**3. Risk Management:**
- Identify risks early (Sensor failures, network issues)
- Mitigation strategies
- Regular safety audits

**4. Knowledge Transfer:**
- Internal workshops (wie diese Academy!)
- Documentation wiki
- Code reviews
- Pair programming

---
## 8. Abschlusspräsentation - Template

### Präsentations-Struktur (10 Minuten)

**Folie 1: Titel**
- Team-Name
- Challenge-Titel
- Team-Mitglieder

**Folie 2: Problem Statement**
- Was war die Aufgabe?
- Warum ist das wichtig?
- Was waren die Herausforderungen?

**Folie 3: Lösungsansatz**
- Architektur-Diagramm
- Welche Technologien?
- Warum diese Wahl?

**Folie 4: Implementation**
- Kernalgorithmen
- Code-Snippets (max. 10 Zeilen)
- Besondere Tricks/Optimierungen

**Folie 5: Demo**
- Video (1-2 Minuten)
- Oder Live-Demo
- Zeige Erfolge UND Fehler

**Folie 6: Ergebnisse**
- Quantitative Metriken (Erfolgsrate, Laufzeit, ...)
- Qualitative Beobachtungen
- Vergleich zu Baseline/Alternativen

**Folie 7: Lessons Learned**
- Was hat funktioniert?
- Was nicht?
- Was würden Sie anders machen?

**Folie 8: Future Work**
- Nächste Schritte
- Wie zu Produktion skalieren?
- Weitere Anwendungsfälle

**Folie 9: Q&A**

---
## 9. Zusammenfassung: 4-Tage Academy

### Tag 1: Grundlagen
✓ Hardware-Architektur (DOF, Sensoren, Aktuatoren)  
✓ Software-Stack (FSM, SDK, Threading)  
✓ Setup & Netzwerk  
✓ Hanger Boot Sequence  
✓ Erste Bewegungen  

### Tag 2: Bewegungssteuerung
✓ Dynamisches Gehen & Stabilität  
✓ PID-Regelung  
✓ Reinforcement Learning  
✓ Keyboard-Tele-Operation  
✓ Trajektorien-Planung  

### Tag 3: Navigation
✓ SLAM (KISS-ICP)  
✓ LiDAR-Integration  
✓ Globale Pfadplanung (A*, RRT)  
✓ Lokale Hindernisvermeidung (DWA)  
✓ RealSense RGB-D  

### Tag 4: Wahrnehmung & Integration
✓ Objekterkennung (YOLO)  
✓ Semantisches Mapping  
✓ Multi-Sensor-Fusion  
✓ Geoff-Stack Integration  
✓ Real-World Challenge  
✓ Transfer ins Unternehmen  

### Wichtigste Takeaways:
1. **Modularität**: Komponenten austauschbar halten
2. **Robustheit**: Fehlerbehandlung von Anfang an
3. **Testing**: Unit + Integration Tests essentiell
4. **Monitoring**: Logging & Metrics für Produktion
5. **Iteration**: Von einfach zu komplex, schrittweise

---
## 10. Weiterführende Ressourcen

### Dokumentation
- **Unitree G1 Docs**: `docs/` Verzeichnis im Repository
- **KISS-ICP Paper**: https://github.com/PRBonn/kiss-icp
- **YOLO Docs**: https://docs.ultralytics.com/

### Kurse & Tutorials
- **Modern Robotics** (Northwestern): Comprehensive textbook + videos
- **Probabilistic Robotics** (Thrun et al.): SLAM bible
- **Deep RL Course** (Hugging Face): Für RL-Vertiefung

### Communities
- **ROS Discourse**: https://discourse.ros.org/
- **Reddit r/robotics**: Active community
- **Unitree Forum**: Official support

### Tools
- **MuJoCo**: Simulation (bereits im Repo)
- **Gazebo**: Alternative Simulation
- **RViz**: Visualisierung (wenn ROS verwendet)
- **Weights & Biases**: ML Experiment Tracking

---
## 11. Abschlusswort

**Herzlichen Glückwunsch!** Sie haben die 4-tägige Unitree G1 Academy abgeschlossen.

Sie haben gelernt:
- Wie man einen humanoiden Roboter steuert
- Wie man autonom navigiert
- Wie man Objekte erkennt und manipuliert
- Wie man alles zu einem System integriert
- Wie man es in Produktion bringt

**Nächste Schritte:**
1. Experimentieren Sie weiter mit dem Code
2. Erweitern Sie die Funktionalität
3. Teilen Sie Ihre Ergebnisse
4. Starten Sie eigene Projekte!

**Danke für Ihre Teilnahme!**

**Fragen? Kontakt:**
- EF Robotics: info@ef-robotics.com
- GitHub Issues: (Repository URL)
- Slack/Discord: (Community Link)

---

*Happy Hacking! Viel Erfolg mit Ihren Robotik-Projekten!*